<a href="https://colab.research.google.com/github/kimjeongsoo20190147/bank_transaction_fraud_detation/blob/main/bank_transaction_fraud_detaction_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import os

import numpy as np
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb


In [14]:
print("XGBoost version:", xgb.__version__)

XGBoost version: 2.1.4


In [15]:
##############################
# 1) 데이터 로드
##############################
train_transaction = pd.read_csv('/content/train_transaction.csv', index_col='TransactionID')
test_transaction  = pd.read_csv('/content/test_transaction.csv',  index_col='TransactionID')

train_identity    = pd.read_csv('/content/train_identity.csv',    index_col='TransactionID')
test_identity     = pd.read_csv('/content/test_identity.csv',     index_col='TransactionID')

sample_submission = pd.read_csv('/content/sample_submission.csv', index_col='TransactionID')

##############################
# 2) 컬럼명 치환 ( '-' → '_' )
##############################
train_transaction.columns = [col.replace('-', '_') for col in train_transaction.columns]
test_transaction.columns  = [col.replace('-', '_') for col in test_transaction.columns]
train_identity.columns     = [col.replace('-', '_') for col in train_identity.columns]
test_identity.columns      = [col.replace('-', '_') for col in test_identity.columns]

##############################
# 3) Train / Test 병합
##############################
# train_transaction + train_identity
train = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)

# test_transaction + test_identity
test  = test_transaction.merge(test_identity,   how='left', left_index=True, right_index=True)

del train_transaction, train_identity, test_transaction, test_identity


print("Train shape:", train.shape)
print("Test  shape:", test.shape)
# 예: Train (590540, 433), Test (506691, 432) 등

##############################
# 4) 타깃 분리, 불필요 DF 삭제
##############################
y_train = train['isFraud'].copy()

X_train = train.drop('isFraud', axis=1)
X_test  = test.copy()

del train, test


##############################
# 5) 결측치 처리
##############################
X_train = X_train.fillna(-999)
X_test  = X_test.fillna(-999)

##############################
# 6) 라벨 인코딩
##############################
for f in X_train.columns:
    if X_train[f].dtype=='object' or X_test[f].dtype=='object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f].values) + list(X_test[f].values))
        X_train[f] = lbl.transform(list(X_train[f].values))
        X_test[f]  = lbl.transform(list(X_test[f].values))

print("X_train shape:", X_train.shape)
print("X_test  shape:", X_test.shape)
# 여기서 X_test.shape[0]가 506,691이어야 함


Train shape: (590540, 433)
Test  shape: (506691, 432)
X_train shape: (590540, 432)
X_test  shape: (506691, 432)


In [16]:
clf = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=9,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    missing=-999,
    random_state=2019,
    tree_method='gpu_hist'  # THE MAGICAL PARAMETER
)

In [17]:
%time clf.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:45:11] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


CPU times: user 1min 29s, sys: 1.39 s, total: 1min 31s
Wall time: 34.9 s


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=9, max_leaves=None,
              min_child_weight=None, missing=-999, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, random_state=2019, ...)

In [18]:
# 8) 예측 후 제출
##############################
sample_submission['isFraud'] = clf.predict_proba(X_test)[:, 1]
sample_submission.to_csv('simple_xgboost.csv')

print("Done! Saved to simple_xgboost.csv")

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:45:41] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Done! Saved to simple_xgboost.csv
